I chose Task 2 because it aligns with my expertise in geospatial analysis and GEE. 

I will use GEEMAP which is a python package for inyteractive mapping with Google Earth Engine

In [1]:
import ee
import geemap
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
import pandas as pd

In [2]:
# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Used the coordinates that were provided:

In [3]:
# Define AOI
AOI = ee.Geometry.Polygon([
    [
        [36.306872504019424, -0.5952725839718056],
        [36.306872504019424, -0.7423752404067301],
        [36.45206844566468, -0.7423752404067301],
        [36.45206844566468, -0.5952725839718056],
        [36.306872504019424, -0.5952725839718056]
    ]
])

In [4]:
# Create map and adding our region of intrest
Map = geemap.Map()
Map.centerObject(AOI, 12)
Map.addLayer(AOI, {'color': 'red'}, 'AOI')
Map

Map(center=[-0.6688240816217108, 36.37947047484175], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
# Define time range (last 3 months)
end_date = ee.Date("2025-02-28")
start_date = end_date.advance(-3, 'month')

In [6]:
# Fetch and clip Sentinel-1 VV and VH
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(AOI)\
    .filterDate(start_date, end_date)\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .select(['VV', 'VH'])\
    .mean()\
    .clip(AOI)  # Clip to AOI

Map.addLayer(s1.select('VV'), {'min': -25, 'max': 0}, 'Sentinel-1 VV')
Map.addLayer(s1.select('VH'), {'min': -25, 'max': 0}, 'Sentinel-1 VH')
Map

Map(center=[-0.6688240816217108, 36.37947047484175], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
# Fetch and clip Sentinel-2 NDVI
s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(AOI)\
    .filterDate(start_date, end_date)\
    .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))\
    .mean()\
    .clip(AOI)  # Clip to AOI

Map.addLayer(s2, {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'Sentinel-2 NDVI')
Map

Map(center=[-0.6688240816217108, 36.37947047484175], controls=(WidgetControl(options=['position', 'transparent…

In [8]:
# Fetch and clip ERA5 Temperature Data
era5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
    .filterDate('2023-11-01', '2024-02-01')\
    .select('temperature_2m')\
    .map(lambda img: img.set('system:time_start', img.date().millis()))\
    .reduce(ee.Reducer.mean())\
    .clip(AOI)  # Clip to AOI

Map.addLayer(era5, {'min': 280, 'max': 310, 'palette': ['blue', 'purple', 'red']}, 'Temperature')
Map

Map(center=[-0.6688240816217108, 36.37947047484175], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
# Load SRTM Elevation
elevation = ee.Image('USGS/SRTMGL1_003').clip(AOI)
Map.addLayer(elevation, {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'brown']}, 'Elevation')

# Display the map
Map

Map(center=[-0.6688240816217108, 36.37947047484175], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
def resample_to_10m(img):
    s2_proj = ee.ImageCollection('COPERNICUS/S2_SR').first().select('B4').projection()  # Get projection from first image
    return img.resample('bilinear').reproject(crs=s2_proj, scale=10)

s1 = resample_to_10m(s1)
s2 = resample_to_10m(s2)
era5 = resample_to_10m(era5)
elevation = resample_to_10m(elevation)

In [11]:
print(type(s1))
print(type(s2))
print(type(era5))
print(type(elevation))

<class 'ee.image.Image'>
<class 'ee.image.Image'>
<class 'ee.image.Image'>
<class 'ee.image.Image'>


In [12]:
print('S1 Projection:', s1.projection().getInfo())
print('S2 Projection:', s2.projection().getInfo())
print('ERA5 Projection:', era5.projection().getInfo())
print('Elevation Projection:', elevation.projection().getInfo())

S1 Projection: {'type': 'Projection', 'crs': 'EPSG:32635', 'transform': [10, 0, 499980, 0, 10, 3000000]}
S2 Projection: {'type': 'Projection', 'crs': 'EPSG:32635', 'transform': [10, 0, 499980, 0, 10, 3000000]}
ERA5 Projection: {'type': 'Projection', 'crs': 'EPSG:32635', 'transform': [10, 0, 499980, 0, 10, 3000000]}
Elevation Projection: {'type': 'Projection', 'crs': 'EPSG:32635', 'transform': [10, 0, 499980, 0, 10, 3000000]}


In [13]:
print(s1.bandNames().getInfo())
print(s2.bandNames().getInfo())
print(era5.bandNames().getInfo())
print(elevation.bandNames().getInfo())

['VV', 'VH']
['NDVI']
['temperature_2m_mean']
['elevation']


In [14]:
# Stack all images into a single multi-band image (datacube)
datacube = ee.Image.cat([s1, s2, era5, elevation]).rename([
    'S1_VV', 'S1_VH',  # 2 bands from Sentinel-1
    'S2_NDVI',  # 1 band from Sentinel-2
    'ERA5_Temperature',  # 1 band from ERA5
    'Elevation'  # 1 band from Elevation
])

# Print information about the final datacube
print(datacube.getInfo())

{'type': 'Image', 'bands': [{'id': 'S1_VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [22592, 22451], 'origin': [89259, -322450], 'crs': 'EPSG:32635', 'crs_transform': [10, 0, 499980, 0, 10, 3000000]}, {'id': 'S1_VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [22592, 22451], 'origin': [89259, -322450], 'crs': 'EPSG:32635', 'crs_transform': [10, 0, 499980, 0, 10, 3000000]}, {'id': 'S2_NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [22592, 22451], 'origin': [89259, -322450], 'crs': 'EPSG:32635', 'crs_transform': [10, 0, 499980, 0, 10, 3000000]}, {'id': 'ERA5_Temperature', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [22592, 22451], 'origin': [89259, -322450], 'crs': 'EPSG:32635', 'crs_transform': [10, 0, 499980, 0, 10, 3000000]}, {'id': 'Elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': 

In [15]:
print(datacube.bandNames().getInfo())

['S1_VV', 'S1_VH', 'S2_NDVI', 'ERA5_Temperature', 'Elevation']


In [16]:
# Convert all bands to Float32
datacube = datacube.toFloat()

In [17]:
task = ee.batch.Export.image.toDrive(
    image=datacube,
    description='Unified_Datacube',
    folder='EarthEngine_Exports',
    fileNamePrefix='datacube',
    region=datacube.geometry(),
    scale=10,  # Use the highest resolution
    crs='EPSG:32635',  # Ensure consistent CRS
    fileFormat='GeoTIFF'
)
task.start()

In [18]:
import time

while task.active():
    print("Export is still running...")
    time.sleep(30)  # Check every 30 seconds

print("Final Status:", task.status())

Export is still running...
Export is still running...
Export is still running...
Export is still running...
Export is still running...
Export is still running...
Export is still running...
Export is still running...
Final Status: {'state': 'COMPLETED', 'description': 'Unified_Datacube', 'priority': 100, 'creation_timestamp_ms': 1740738657940, 'update_timestamp_ms': 1740738896991, 'start_timestamp_ms': 1740738669989, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1QPcSpY-oTJMF4NpdErmf6XpcBQW6hk3Q'], 'attempt': 1, 'batch_eecu_usage_seconds': 468.0362854003906, 'id': '63M22O4XIEACCPYLL4LTLT5U', 'name': 'projects/earthengine-legacy/operations/63M22O4XIEACCPYLL4LTLT5U'}


Once downloaded i read and convert to NetCDF 

In [19]:
import rioxarray as rxr

# Open the raster dataset
datacube_xr = rxr.open_rasterio("datacube.tif")

# Print dataset details
print(datacube_xr)
print(datacube_xr.attrs)  # Check for any non-standard attributes

<xarray.DataArray (band: 5, y: 1653, x: 1642)> Size: 54MB
[13571130 values with dtype=float32]
Coordinates:
  * band         (band) int32 20B 1 2 3 4 5
  * x            (x) float64 13kB 1.54e+06 1.54e+06 ... 1.557e+06 1.557e+06
  * y            (y) float64 13kB -6.668e+04 -6.668e+04 ... -8.318e+04 -8.32e+04
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('S1_VV', 'S1_VH', 'S2_NDVI', 'ERA5_Temperature', 'Elevat...
{'AREA_OR_POINT': 'Area', 'scale_factor': 1.0, 'add_offset': 0.0, 'long_name': ('S1_VV', 'S1_VH', 'S2_NDVI', 'ERA5_Temperature', 'Elevation')}


In [24]:
# Remove problematic attributes
datacube_xr.attrs = {}  # Clears global attributes
for var in datacube_xr.variables:
    datacube[var].attrs = {}  # Clears attributes for each variable

# Save again
datacube_xr.to_netcdf("geospatial_datacube.nc")

In [25]:
ds = xr.open_dataset("geospatial_datacube.nc")
print(ds)
ds.close()

<xarray.Dataset> Size: 54MB
Dimensions:      (y: 1653, x: 1642, band: 5)
Coordinates:
  * y            (y) float64 13kB -6.668e+04 -6.668e+04 ... -8.318e+04 -8.32e+04
  * x            (x) float64 13kB 1.54e+06 1.54e+06 ... 1.557e+06 1.557e+06
  * band         (band) int32 20B 1 2 3 4 5
Data variables:
    datacube     (band, y, x) float32 54MB ...
    spatial_ref  int32 4B ...


DONE!